In [ ]:
# hide
# dont_test
# default_exp handlers
%load_ext nb_black

<IPython.core.display.Javascript object>

# ASGIHandlers

In [ ]:
# export

from asgiref.sync import sync_to_async
from django.core.handlers.asgi import ASGIHandler


class AsyncFileASGIHandler(ASGIHandler):
    async def send_response(self, response, send):
        if response.is_async:
            return await response.stream(send)
        else:
            return super().send_response(response, send)

<IPython.core.display.Javascript object>

## Tests

In [ ]:
class Response:
    is_async = True
    stream_called = False

    def __init__(self, content):
        self.content = content

    async def stream(self, send):
        self.stream_called = True
        return await send(self.content)


async def send(data):
    return data

<IPython.core.display.Javascript object>

In [ ]:
%env

{'PWD': '/Users/jochen/projects/django_fileresponse/nbs',
 'FZF_DEFAULT_COMMAND': 'fd --type f',
 'PYENV_ROOT': '/Users/jochen/.pyenv',
 'VIRTUAL_ENV': '/Users/jochen/.virtualenvs/django-fileresponse',
 'PYENV_SHELL': 'fish',
 'OMF_PATH': '/Users/jochen/.local/share/omf',
 'LDFLAGS': '-L/usr/local/opt/openssl@1.1/lib',
 'HOME': '/Users/jochen',
 'PKG_CONFIG_PATH': '/usr/local/opt/openssl@1.1/lib/pkgconfig',
 'CPATH': '/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include:/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include',
 '__CF_USER_TEXT_ENCODING': '0x1F5:0x0:0x3',
 'XPC_SERVICE_NAME': '0',
 'LC_TERMINAL_VERSION': '3.4.8',
 'SHELL': '/usr/local/bin/fish',
 '__CFBundleIdentifier': 'com.googlecode.iterm2',
 'TERM_SESSION_ID': 'w0t1p1:C06A30C0-D3A6-4160-8E3D-26F4CE6DE5A4',
 'TERM_PROGRAM_VERSION': '3.4.8',
 'TERM_PROGRAM': 'iTerm.app',
 'COMPOSE_DOCKER_CLI_BUILD': '1',
 'ITERM_PR

<IPython.core.display.Javascript object>

In [ ]:
%env DJANGO_SETTINGS_MODULE example.settings
import django

django.setup()
app = AsyncFileASGIHandler()
content = "<h1>Content!</h1>"
response = Response(content)
returned_content = await app.send_response(response, send)
assert response.stream_called
assert content == returned_content

env: DJANGO_SETTINGS_MODULE=example.settings


<IPython.core.display.Javascript object>